<img src="https://raw.githubusercontent.com/DonAurelio/open-datacube-bac-training/main/docs/banner.png" alt="Deparatemento de Ingeniería de Sistemas y Computación, Universidad de los Andes">

#  Análisis espacio temporal - Aplicación de algoritmos para el análisis temporal

**Introducción**

Los compuestos temporales reducen la incidencia de las nubes en imágenes de satélite. Para ello se realiza un proceso de selección de pixeles en una serie de tiempo para una imagen. Como resultado del proceso de selección, se obtiene una imagen o composición que contiene pixeles de diferentes periodos de tiempo. Estos píxeles son seleccionados mediante la aplicación de un criterio de selección de pixeles. Algunos métodos de selección de píxeles conocidos son: Most Recent / Least Recent Pixel, Maximum Value Composite (MVC), Median Composite, Geomedian Composite, Best Pixel, entre otros. 

En el presente notebook exploraremos uno de los compuestos mas conocidos, el compuesto de medianas y también haremos un ejercicio sencillo de detección de cambio

**Precondiciones - Actividad previa**

1. Tener dos imágenes en el cubo - Ver notebooks 1 y 2
2. Haber realizado las actividades de los notebooks 3, 4 y 5

**Contenido**

1. Importar librerías
2. Definición del área de estudio y consulta de las imágenes
3. Cálculo del compuesto de medianas
4. Detección de cambio
5. Guardar resultados de análisis en formato netcdf y en tiff

**Nota importante:** Este notebook requiere ya de mucha memoria del computador. En caso de presentarse errores de memoria tome las siguientes acciones:
- Reinicie el Kernel
- En las celdas que dibujan las imágenes, quite los parámetros `size` y `aspect`. eso hace que la visualización de la respuesta sea más pequeña, que consuma menos memoria y que sea más rápida la ejecución de las celdas
- Disminuya el tamaño de la zona de estudio, cambiando el valor de la variable buffer


## 1. Importar librerías
___
En esta sección se importan las librerías cuya funcionalidades particulares son requeridas.

In [ ]:
# las funcionalidades del open data cube son accedidas 
# por medio de la librería datacube
import datacube

# Librería para cálculo en matrices
import numpy as np

# Manipulación de datasets
import xarray as xr

# Manipulación de datos raster
import rasterio

# Librería usada para la carga de polígonos
import geopandas as gpd

# Librería usada para visualización de datos
import matplotlib as mpl
import matplotlib.pyplot as plt

# Desactiva los warnings en el notebook
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

## 2. Definición del área de estudio y consulta de las imágenes
___
Los coordenadas del punto a seleccionar pueden ser obtenidas a través de herramientas GIS como Google maps. Este punto debe estar comprendido en el área que desea estudiar. El punto definido es empleado para la generación de un cuadrado que finalmente será usado para consultar el área de estudio. La variable `buffer` permite ampliar o disminuir las dimensiones del cuadrado. Lo anterior es equivalente a disminuir o ampliar el área de estudio a consultar en el open data cube.

<img src="https://raw.githubusercontent.com/DonAurelio/open-datacube-bac-training/main/docs/latlong_buffer.png" alt="Definición área de estudio" width="20%">

In [ ]:
# Definición de las coordenadas del punto
central_lat = 5.547964746532565
central_lon =  -72.9284962779124

# Aumento del aŕea del cuadrado para "EPSG:4326" (WGS84 - Unidades en grados)
#  0.1 grados que corresponden a 11.1 kilómetros alrededor del punto de interés
buffer = 0.1

# Calculo del cuadro delimitador (bounding box) para el área de estudio
set_study_area_lat = (central_lat - buffer, central_lat + buffer)
set_study_area_lon = (central_lon - buffer, central_lon + buffer)

print(f'    latitude={set_study_area_lat},')
print(f'    longitude={set_study_area_lon},')

**Ahora recuperamos las imágenes que tenga el cubo y que contienen el área de interés**

In [ ]:
dc = datacube.Datacube(app="MOOC GEO")

dataset = dc.load(
    product="s2_sen2cor_ard_granule_EO3",
    latitude=(5.447964746532565, 5.647964746532565),
    longitude=(-73.0284962779124, -72.82849627791241),
    time=('2021-01-01', '2021-02-01'),
    measurements=["red","blue","green","nir","swir1","swir2","scl"],
    crs="EPSG:4326",
    output_crs="EPSG:4326",
    resolution=(-0.00008983111,0.00008971023)
)

# Ver el resultado de la consulta, en el formato del cubo
dataset

**También queremos ver las imágenes recuperadas**

In [ ]:
rgb = dataset[["red","green","blue"]].to_array(dim='color')
rgb = rgb.transpose(*(rgb.dims[1:]+rgb.dims[:1]))  # make 'color' the last dimension
img = rgb.plot.imshow(col='time',col_wrap=2,add_colorbar=False,vmin=0,vmax=1200, size = 5, aspect=1)

## 3. Cálculo del compuesto de medianas
___
El compuesto de medianas (Median Composite), es precisamente el cálculo de la mediana  de  cada  píxel  en  una  serie  de  tiempo. 

<img src="https://raw.githubusercontent.com/DonAurelio/open-datacube-bac-training/main/docs/composite.png" alt="Compuesto de medianas" width="60%">

El cálculo del compuesto de medianas comprende dos pasos: 

1. **Aplicación de la máscara de nubes**
2. **Generación del compuesto**

### Funciones auxiliares para el cálculo del compuesto de medianas
Las siguientes son algunas funciones que ayudan a generar la máscara de nubes, apoyadas en la banda de calidad de Sentinel-2, que ya veremos más en detalle

In [ ]:
def unpack_bits(land_cover_endcoding, data_array, cover_type):
    """
    Description:
        Unpack bits for end of ls7 and ls8 functions 
    -----
    Input:
        land_cover_encoding(dict hash table) land cover endcoding provided by ls7 or ls8
        data_array( xarray DataArray)
        cover_type(String) type of cover
    Output:
        unpacked DataArray
    """
    data = data_array.data
    if isinstance(data, np.ndarray):
        boolean_mask = np.isin(data, land_cover_endcoding[cover_type]) 
    elif isinstance(data, dask.array.core.Array):
        boolean_mask = dask.array.isin(data, land_cover_endcoding[cover_type])
    return xr.DataArray(boolean_mask.astype(bool),
                        coords = data_array.coords,
                        dims = data_array.dims,
                        name = cover_type + "_mask",
                        attrs = data_array.attrs)

def s2_unpack_qa(data_array , cover_type):

    land_cover_endcoding = dict( 
        no_data                 =[0],
        saturated_or_defective  =[1],
        dark_area               =[2],
        cloud_shadow            =[3],
        vegetation              =[4],
        not_vegetated           =[5],
        water                   =[6],
        unclassified            =[7],
        cloud_medium_prob       =[8],
        cloud_high_prob         =[9],
        thin_cirrus             =[10],
        snow                    =[11]
    )
    return unpack_bits(land_cover_endcoding, data_array, cover_type)

def qa_clean_mask(dataset, platform, cover_types=['vegetation', 'not_vegetated']):
    """
    Returns a clean_mask for `dataset` that masks out various types of terrain cover using the
    Landsat pixel_qa band. Note that Landsat masks specify what to keep, not what to remove.
    This means that using `cover_types=['clear', 'water']` should keep only clear land and water.

    See "pixel_qa band" here: https://landsat.usgs.gov/landsat-surface-reflectance-quality-assessment
    and Section 7 here: https://landsat.usgs.gov/sites/default/files/documents/lasrc_product_guide.pdf.

    Parameters
    ----------
    dataset: xarray.Dataset
        An xarray (usually produced by `datacube.load()`) that contains a `pixel_qa` data
        variable.
    platform: str
        A string denoting the platform to be used. Can be "LANDSAT_5", "LANDSAT_7", or
        "LANDSAT_8".
    cover_types: list
        A list of the cover types to include. Adding a cover type allows it to remain in the masked data.
        Cover types for all Landsat platforms include:
        ['fill', 'clear', 'water', 'shadow', 'snow', 'cloud', 'low_conf_cl', 'med_conf_cl', 'high_conf_cl'].

        'fill' removes "no_data" values, which indicates an absense of data. This value is -9999 for Landsat platforms.
        Generally, don't use 'fill'.
        'clear' allows only clear terrain. 'water' allows only water. 'shadow' allows only cloud shadows.
        'snow' allows only snow. 'cloud' allows only clouds, but note that it often only selects cloud boundaries.
        'low_conf_cl', 'med_conf_cl', and 'high_conf_cl' denote low, medium, and high confidence in cloud coverage.
        'low_conf_cl' is useful on its own for only removing clouds, however, 'clear' is usually better suited for this.
        'med_conf_cl' is useful in combination with 'low_conf_cl' to allow slightly heavier cloud coverage.
        Note that 'med_conf_cl' and 'cloud' are very similar.
        'high_conf_cl' is useful in combination with both 'low_conf_cl' and 'med_conf_cl'.

        For Landsat 8, there are more cover types: ['low_conf_cir', 'high_conf_cir', 'terrain_occ'].
        'low_conf_cir' and 'high_conf_cir' denote low and high confidence in cirrus clouds.
        'terrain_occ' allows only occluded terrain.

    Returns
    -------
    clean_mask: xarray.DataArray
        An xarray DataArray with the same number and order of coordinates as in `dataset`.
    """
    processing_options = {
        "SENTINEL_2": s2_unpack_qa
    }

    clean_mask = None
    # Keep all specified cover types (e.g. 'clear', 'water'), so logically or the separate masks.
    for i, cover_type in enumerate(cover_types):
        cover_type_clean_mask = processing_options[platform](dataset.scl, cover_type)
        clean_mask = cover_type_clean_mask if i == 0 else (clean_mask | cover_type_clean_mask)
    return clean_mask

### Generación de la máscara de nubes

Durante la preparación de las imágenes satelitales Sentinel 2 se genera una banda de calidad llamada `scl`, que es una de las bandas solicitadas al cubo al hacer la consulta del área de estudio. 

Esta banda de calidad es el resultado de la ejecución de varíos algoritmos especializados de detección de coberturas específicas. 

Más información sobre estos algoritmos se muestra [aquí](https://sentinel.esa.int/web/sentinel/technical-guides/sentinel-2-msi/level-2a/algorithm). 

La tabla a continuación, presenta los valores de la banda de calidad `scl`. Con base en esta banda se aplica el algoritmo de enmascaramiento de nubes (remover las nubes). 

| Atributo                 | Valor del Píxel |
|--------------------------|-----------------|
| No Data                  | 0               |
| Saturated or defective   | 1               |
| Dark area Pixels         | 2               |
| Cloud Shadows            | 3               |
| Vegetation               | 4               |
| Not vegetated            | 5               |
| Water                    | 6               |
| Unclassified             | 7               |
| Cloud Medium probability | 8               |
| Cloud High probability   | 9               |
| Thin Cirrus              | 10              |
| Snow                     | 11              |

**Lo primero que haremos es visualizar esta banda, con la clasificación aquí explicada**

In [ ]:
# Definición de la paleta de colores
cmap = mpl.colors.ListedColormap(
    [
        '#000000', 
        '#fe0000',
        '#404040', 
        '#833d0c', 
        '#00ff01', 
        '#ffff01', 
        '#0000cc', 
        '#757170', 
        '#aeaaa9', 
        '#d0ced0', 
        '#00ccff', 
        '#ff66ff'
    ]
)

# Rango de valores establecidos para cada color
bounds = [0,1,2,3,4,5,6,7,8,9,10,11,12]

# Genera una capa de normalización de los datos basada en los intervalos establecidos en 'bounds'
norm = mpl.colors.BoundaryNorm(bounds, cmap.N)

# Generación de la imagen
img = dataset.scl.plot(col='time',col_wrap=2, cmap=cmap, norm=norm, size = 5)

#Generación de la leyenda de la imagen, que ayuda a la comprensión de la misma
classification_labels = [
    'no_data',
    'saturated_defective',
    'dark_area_pixels',
    'cloud_shadows',
    'vegetation',
    'not_vegetated',
    'water',
    'unclassified',
    'cloud_medium_probability',
    'Cloud_high_probability',
    'thin_cirrus',
    'snow'
]

# Permite centrar las etiquetas de los colores en el colorbar
classification_labels_ticks = np.linspace(0.5, 11.5, num=12)

# Configuración de las etiquetas de la barra de colores
img.cbar.set_ticklabels(classification_labels)
img.cbar.set_ticks(classification_labels_ticks)

En nuestro caso particular, estamos interesados en los píxeles que presentan probabilidades altas de ser vegetación. Por lo tanto en la máscara de nubes indicamos que deseamos conservar estos píxeles (`qa_clean_mask(dataset,platform='SENTINEL_2',cover_types=['vegetation'])`).

In [ ]:
# Generación máscara para valores inválidos
mask_nan = ~np.isnan(dataset)              # Deja los píxeles que tienen valor diferente a NaN
mask_inf = ~np.isinf(dataset)              # Deja los píxeles que tienen valor diferente a +/- inifinito

# Generación máscara de nubes
clean_mask = qa_clean_mask(dataset,platform='SENTINEL_2',cover_types=['vegetation'])

# La máscara con el formato del cubo
clean_mask

Observamos que la máscara, para cada imagen, es una matriz de valores booleanos (True, False) que indica, para cada pixel, si en esa posición hay un valor válido para análisis.

**Miremos también la imagen de la máscara de vegetación**

Para cada una de las imágenes, obtenemos en amarillo los píxeles que son válidos para análisis de vegetación y en morado los que no

In [ ]:
# Generación de la imagen de la máscara
img = clean_mask.plot(col='time',col_wrap=2, add_colorbar=False, size = 5, aspect = 1)

In [ ]:
# Aplicación de la máscara de nubes
clean_dataset = dataset.where(clean_mask & mask_nan & mask_inf)

# Visualización en verdadero color de las imágenes, a las cuales se les eliminó, 
# por medio de la máscara: los píxeles inválidos y los que no están clasificados como vegetación
rgb = clean_dataset[["red","green","blue"]].to_array(dim='color')
rgb = rgb.transpose(*(rgb.dims[1:]+rgb.dims[:1]))  # make 'color' the last dimension
img = rgb.plot.imshow(col='time',col_wrap=2,add_colorbar=False,vmin=0, vmax=1200, size=5, aspect = 1)

### Cálculo del compuesto de medianas 
Ya teniendo las dos imágenes limpias, podemos proceder al cálculos de las medianas.

Primero vemos el resultado en la estructura de datos y luego la visualizamos en verdadero color

In [ ]:
median_composite = clean_dataset.median('time', skipna=True)
median_composite

In [ ]:
rgb = median_composite[["red","green","blue"]].to_array(dim='color')
rgb = rgb.transpose(*(rgb.dims[1:]+rgb.dims[:1]))  # make 'color' the last dimension
img = rgb.plot.imshow(add_colorbar=False,vmin=0,vmax=1200,figsize=(10,10))

En el resultado todavía hay mucha cobertura de nubes. Esto es debido a que sólo estamos utilizando dos imágenes...

Para lograr una imagen sin nubes, es necesario procesar decenas de imágenes del mismo sitio y componerlas, tomando los píxeles que tienen valores válidos. Esto se llama armar un mosaico de imágenes.

## 4. Detección de cambio
___
Una de las preguntas más frecuentes es saber qué cambió en una región entre dos momentos del tiempo. Hay muchas formas de responder esta pregunta, que no es nada evidente...

En este ejercicio vamos a responder una pregunta más sencilla: ¿Dónde hubo cambios del NVDI entre estos dos momentos del tiempo?

Y la forma de responderla es trabajando sobre las imágenes limpias y simplemente restando el NVDI de una imagen del de la otra. 

Resultados cercanos a 0 (en gris) se interpretan como que no hubo cambio y valores lejanos de 0 se interpretan como que algo cambió (en rojo y azul).

In [ ]:
# Cálculo del NDVI de las imágenes limpias
clean_dataset['ndvi'] = (clean_dataset.nir - clean_dataset.red) / (clean_dataset.nir + clean_dataset.red)

# Cálculo de la diferencia del NDVI de las dos imágenes
ndvi_diff = clean_dataset['ndvi'].isel(time = 1) - clean_dataset['ndvi'].isel (time = 0)
ndvi_diff

In [ ]:
# Visualización del resultado

# Definición de la paleta de colores
cmap = mpl.colors.ListedColormap(
    [
        '#0000FF',
        '#f1c40f',
        '#D8D8D8',
        '#b03a2e', 
        '#3ADF00' 
    ]
)

# Rango de valores establecidos para cada color
bounds = [-1,-0.6,-0.1,0.1,0.6,1]

# Genera una capa de normalización de los datos basada en los intervalos establecidos en 'bounds'
norm = mpl.colors.BoundaryNorm(bounds, cmap.N)

#Generación de la imagen
ndvi_diff.plot (size=10, aspect=1, cmap = cmap, norm=norm)

**Aunque este es un resultado relativamente simple y que necesita más elaboración para la resolución de algún problema particular, ilustra la potencia y versatibilidad de trabajar con el cubo de datos**

## 5. Guardar resultados de análisis en formato netcdf y en tiff
___
Al igual que antes, podemos guardar los resultados para análisis posteriores con herramientas SIG.

Estas celdas están programadas para guardar los resultados del cálculo de medianas, pero pueden guardar cualquiera (o todos) los resultados de los ejercicios de este notebook

### Guardar resultados de análisis en un archivo .nc

In [ ]:
median_composite.to_netcdf('Salidas/mediana.nc')

### Guardar resultados de análisis en formato geotiff
___
Funciones requeridas para guardar información en geotiff

In [ ]:
"""
Las funciones mostradas a continuación fueron tomadas de 
https://github.com/ceos-seo/data_cube_notebooks
"""

def _get_transform_from_xr(data, x_coord='longitude', y_coord='latitude'):
    """Create a geotransform from an xarray.Dataset or xarray.DataArray.
    """

    from rasterio.transform import from_bounds
    geotransform = from_bounds(data[x_coord][0], data[y_coord][-1],
                               data[x_coord][-1], data[y_coord][0],
                               len(data[x_coord]), len(data[y_coord]))
    return geotransform

def write_geotiff_from_xr(tif_path, data, bands=None, no_data=-9999, crs="EPSG:4326",
                          x_coord='longitude', y_coord='latitude'):
    """
    NOTE: Instead of this function, please use `import_export.export_xarray_to_geotiff()`.
    Export a GeoTIFF from an `xarray.Dataset`.
    Parameters
    ----------
    tif_path: string
        The path to write the GeoTIFF file to. You should include the file extension.
    data: xarray.Dataset or xarray.DataArray
    bands: list of string
        The bands to write - in the order they should be written.
        Ignored if `data` is an `xarray.DataArray`.
    no_data: int
        The nodata value.
    crs: string
        The CRS of the output.
    x_coord, y_coord: string
        The string names of the x and y dimensions.
    """
    if isinstance(data, xr.DataArray):
        height, width = data.sizes[y_coord], data.sizes[x_coord]
        count, dtype = 1, data.dtype
    else:
        if bands is None:
            bands = list(data.data_vars.keys())
        else:
            assrt_msg_begin = "The `data` parameter is an `xarray.Dataset`. "
            assert isinstance(bands, list), assrt_msg_begin + "Bands must be a list of strings."
            assert len(bands) > 0 and isinstance(bands[0], str), assrt_msg_begin + "You must supply at least one band."
        height, width = data.dims[y_coord], data.dims[x_coord]
        count, dtype = len(bands), data[bands[0]].dtype
    with rasterio.open(
            tif_path,
            'w',
            driver='GTiff',
            height=height,
            width=width,
            count=count,
            dtype=dtype,
            crs=crs,
            transform=_get_transform_from_xr(data, x_coord=x_coord, y_coord=y_coord),
            nodata=no_data) as dst:
        if isinstance(data, xr.DataArray):
            dst.write(data.values, 1)
        else:
            for index, band in enumerate(bands):
                dst.write(data[band].values, index + 1)
    dst.close()

Guardar resultados de análisis en un archivo .tif

In [ ]:
write_geotiff_from_xr(
    tif_path='Salidas/mediana.tif', 
    data=median_composite, 
    bands=['red','green','blue',],      # El orden de las bandas es importante para visualizar la imagen en una herramienta GIS
    no_data=-9999, 
    crs="EPSG:4326",
    x_coord='longitude',
    y_coord='latitude'
)